# Using PhysiCOOL with third-party libraries

The `BlackBoxModel` class can be used in combination with other libraries to perform parameter studies and optimization routines that have still not been implemented in PhysiCOOL.

Here, we describe how to couple our approach with the [psweep](https://github.com/elcorto/psweep) library, which can be used to set up and run parameter studies. The PhysiCell project used is the one described in the motility example. 

A Pandas DataFrame is created by `psweep` with the model outputs for each parameter set and some additional information (run_id, run_time...). Results are also stored in a folder called `calc`.

This approach can be used with other libraries, as users see fit.

> **Note for users running this notebook locally:** You will need to install psweep (using pip install)

## Compiling the PhysiCell project

In [6]:
from physicool import optimization as opt

# Compiles the project and creates a black box object for it
opt.compile_project() 

g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_vector.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_mesh.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_microenvironment.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_solvers.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_matlab.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_utilities.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_basic_agent.cpp 
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./BioFVM/BioFVM_MultiCellDS.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -f

./modules/PhysiCell_geometry.cpp: In function ‘bool PhysiCell::load_cells_from_pugixml(pugi::xml_node)’:
./modules/PhysiCell_geometry.cpp:304:9: warning: ignoring return value of ‘int system(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
  304 |   system("sleep 1");
      |   ~~~~~~^~~~~~~~~~~


g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -c ./custom_modules/custom.cpp
g++ -march=native  -O3 -fomit-frame-pointer -mfpmath=both -fopenmp -m64 -std=c++11  -o project BioFVM_vector.o BioFVM_mesh.o BioFVM_microenvironment.o BioFVM_solvers.o BioFVM_matlab.o BioFVM_utilities.o BioFVM_basic_agent.o BioFVM_MultiCellDS.o BioFVM_agent_container.o   pugixml.o PhysiCell_phenotype.o PhysiCell_cell_container.o PhysiCell_standard_models.o PhysiCell_cell.o PhysiCell_custom.o PhysiCell_utilities.o PhysiCell_constants.o PhysiCell_basic_signaling.o PhysiCell_signal_behavior.o  PhysiCell_SVG.o PhysiCell_pathology.o PhysiCell_MultiCellDS.o PhysiCell_various_outputs.o PhysiCell_pugixml.o PhysiCell_settings.o PhysiCell_geometry.o custom.o main.cpp 


main.cpp: In function ‘int main(int, char**)’:
main.cpp:106:8: warning: ignoring return value of ‘int system(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
  106 |  system( copy_command );
      |  ~~~~~~^~~~~~~~~~~~~~~~


make name
make[1]: Entering directory '/home/ines/Documents/GitHub/PhysiCOOL/examples/third_party'

Executable name is project

make[1]: Leaving directory '/home/ines/Documents/GitHub/PhysiCOOL/examples/third_party'


## Running the `psweep` parameter study

In [7]:
from physicool. optimization import PhysiCellBlackBox
from physicool.updaters import CellUpdater, update_motility_values
from physicool.processing import get_final_y_position

import psweep as ps
import shutil

# Compiles the project and creates a black box object for it
# opt.compile_project()    
black_box = PhysiCellBlackBox()

# Define the updater we want to use (change motility data)
updater = CellUpdater(updater_function=update_motility_values,
                     config_path="config/PhysiCell_settings.xml", 
                     cell_definition_name="default")

# Assign the updater and processor to the black box
black_box.updater = updater
black_box.processor = get_final_y_position

def func(pset): 
    """
    Runs the black box model with the values selected by psweep.
    """
    shutil.rmtree("temp", ignore_errors=True)
    metric = black_box.run(pset)
    return {'result': metric}

# Choose parameters
# (Creates a grid [["speed": 1.0, "bias": 0.2],
#                  ["speed": 1.0, "bias": 0.6]])
a = ps.plist('migration_bias', [0.2, 0.6])
b = ps.plist('speed', [1.0])
params = ps.pgrid(a,b)

# Get a DataFrame with all the simulation results
# psweep will run func() with each pair of the params grid
df = ps.run_local(func, params)
df.head()

,migration_bias,speed,_pset_hash,_run_id,_pset_id,_calc_dir,_time_utc,_pset_seq,_run_seq,result,_pset_runtime
0,0.2,1.0,cd3eeb3b2452a69eac62b2d2d76d9f29c1d19be4,03829a88-1b29-4fd4-990a-e195896d77b9,3181be1c-e931-40d4-9129-8d9628673bb7,calc,2023-01-29 23:54:29.291671753,0,0,"[-491.465719040412, -152.6958488711255, -490.8...",2.493991
1,0.6,1.0,5a9a45aab62d3a8fed2acdc73b803e96429a24f4,03829a88-1b29-4fd4-990a-e195896d77b9,32f3b479-2f19-4d27-8eea-fd9826a075b8,calc,2023-01-29 23:54:31.788334370,1,0,"[-236.23958073395102, -107.25965253084702, -25...",2.428913
